In [ ]:
# openai 문서 > REST API 문서 > 거기서 나오는 url로 post요청을 보내서 응답 받고, 그걸 json으로 변환하고 등등 모두 수동적으로도 가능, 하지만 아래 방법이 훨 간단하고 개발자 경험이 좋아.
import openai, json
client = openai.OpenAI()
messages = []

def get_weather(city):
  return "33 degrees celcius"

FUNCTION_MAP = {
  "get_weather": get_weather
}

TOOLS = [
  {
    "type": "function",
    "function": {
      "name": "get_weather",
      "description": "A function to get the weather of a city",
      "parameters": {
        "type": "object",
        "properties": {
          "city": {
            "type": "string",
            "description": "The name of the city to get the weatehr of."
          }
        },
        "required": ["city"]
      }
    }
  }
]

from openai.types.chat import ChatCompletionMessage

def process_ai_response(message: ChatCompletionMessage):
  if message.tool_calls: # tool_calls은 list, 근데 아예 없진 않나봐, 에러 발생해서 길이 세는 부분 삭제
    messages.append({
      "role": "assistant",
      "content": message.content or "",
      "tool_calls": [
        {
          "id": tool_call.id,
          "type": "function",
          "function": {
            "name": tool_call.function.name,
            "arguments": tool_call.function.arguments
          }
        } for tool_call in message.tool_calls
      ]
    })

    for tool_call in message.tool_calls:
      function_name = tool_call.function.name
      arguments = tool_call.function.arguments

      print(f"Calling function: {function_name} with {arguments}")

      # 생긴건 아니어도 그냥 한줄의 string을 python에서 활용하기 위해 dictionary로 바꾸는거
      try:
        arguments = json.loads(arguments)
      except json.JSONDecodeError:
        arguments = {}

      function_to_run = FUNCTION_MAP.get(function_name) # FUNCTION_MAP에서 함수 찾고 

      result = function_to_run(**arguments) # 실행

      print(f"Ran {function_name} with args {arguments} for a result of {result}")

      messages.append(
        {
          "role": "tool",
          "tool_call_id": tool_call.id,
          "name": function_name,
          "content": result
        }
      )

    call_ai()

  else:
    messages.append({ "role": "assistant", "content": message.content })
    print(f"AI: {message.content}")

def call_ai():
  response = client.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=TOOLS)
  process_ai_response(response.choices[0].message)

while True:
  message = input("Send a message to the LLM...")
  if message == "quit" or message == "q":
    break
  else:
    messages.append({
      "role":"user",
      "content": message
    })
    print(f"USER: {message}")
    call_ai()

message

USER: i'm chloe
AI: Hi Chloe! How can I assist you today?
USER: what is the weather in seoul?
Calling function: get_weather with {"city":"Seoul"}
Ran get_weather with args {'city': 'Seoul'} for a result of 33 degrees celcius
AI: The weather in Seoul is currently 33 degrees Celsius. If you need more information or updates, feel free to ask!
USER: 
AI: Is there anything else you would like to know or discuss? I'm here to help!
USER: 
AI: It seems like your message didn't come through. If you have any questions or need assistance, please let me know!
USER: 
AI: If there's nothing specific on your mind, that's okay! Just let me know whenever you have a question or if there's something you'd like to talk about. I'm here to help!
USER: 
AI: It seems there might be some issues with the messages. If you have a question or need assistance, please try sending it again. I'm ready to help!


'q'